# Google flood model

Looking into feasibility of using downstream station

[Station](https://sites.research.google/floods/l/11.385801758733372/4.12261962890625/10/g/hybas_1120760290)

In [5]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [142]:
import os
from pathlib import Path
import datetime

from dotenv import load_dotenv
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [31]:
pio.renderers.default = "notebook"

In [25]:
load_dotenv()
GFH_DATA_DIR = Path(os.environ["GFH_DATA_DIR"])
NOWCAST_DIR = GFH_DATA_DIR / "inputs/historic_nowcasts"
RAW_DIR = Path(os.environ["OAP_DATA_DIR"]) / "public/raw/ner"
ABN_PATH = RAW_DIR / "abn/Données_ Niamey2005_2022.xlsx"
HYBAS_ID = 1120760290

In [254]:
# read data
# google
ggl = pd.read_csv(NOWCAST_DIR / f"hybas_{HYBAS_ID}.csv")
ggl["time"] = pd.to_datetime(ggl["time"])
ggl = ggl.set_index("time")

# abn
abn = pd.read_excel(ABN_PATH, skiprows=[0, 1], index_col=0)
# merge
df = ggl.merge(abn, right_index=True, left_index=True)
df = df.rename(columns={"prediction": "Google"})
df

# process
# find minimum average minimum flow day to shift to seasonyear
min_day = (
    df.groupby(df.index.year).idxmin()["Water Level (cm)"].dt.dayofyear.mean()
)
df["dayofseason"] = df.index.dayofyear - min_day
df["dayofseason"] = (
    df["dayofseason"].apply(lambda x: x + 365 if x < 1 else x).astype(int)
)
df["seasonyear"] = (
    pd.to_datetime(df.index.date) - pd.DateOffset(days=min_day)
).year

df = df.sort_values(["seasonyear", "dayofseason"])

# keep only complete years
df = df[df["seasonyear"] != df["seasonyear"].min()]
df = df[df["seasonyear"] != df["seasonyear"].max()]
display(df)

,Google,Water Level (cm),Discharges (m3/s),dayofseason,seasonyear
2005-06-08,17.845198,175.000,146.700,1,2005
2005-06-09,20.193138,225.000,274.000,2,2005
2005-06-10,22.066128,208.667,227.267,3,2005
2005-06-11,23.343590,201.667,208.333,4,2005
2005-06-12,23.400543,197.000,196.267,5,2005
...,...,...,...,...,...
2021-06-03,4.620172,166.000,53.810,361,2020
2021-06-04,4.677755,164.000,51.190,362,2020
2021-06-05,4.068634,162.500,49.265,363,2020
2021-06-06,4.076122,163.000,49.900,364,2020


In [256]:
fig = px.line(
    x=df["dayofseason"],
    y=df["Google"],
    color=df["seasonyear"],
)
fig.add_trace(go.Scatter(x=[125, 125], y=[0, 1000], mode="lines"))
fig.update_traces(line_width=1)
fig.update_layout(template="simple_white")
fig.show()

In [257]:
# find first peak flow date
# cutoff based on eyeballing previous plot
cutoff_dayofseason = 125
df_first_peak = df[df["dayofseason"] < cutoff_dayofseason]
df_max = df_first_peak.groupby("seasonyear").max()
df_max_date = df_first_peak.groupby(df_first_peak["seasonyear"]).idxmax()
df_max_dayofyear = df_max_date.applymap(lambda x: x.dayofyear)
px.scatter(df_max_dayofyear, x="Google", y="Water Level (cm)").show()
display(df_max_dayofyear.corr())
px.scatter(df_max, x="Google", y="Water Level (cm)").show()
display(df_max.corr())

,Google,Water Level (cm),Discharges (m3/s),dayofseason
Google,1.000000,-0.449742,-0.449742,NaN
Water Level (cm),-0.449742,1.000000,1.000000,NaN
Discharges (m3/s),-0.449742,1.000000,1.000000,NaN
dayofseason,NaN,NaN,NaN,NaN


,Google,Water Level (cm),Discharges (m3/s),dayofseason
Google,1.000000,0.533234,0.389201,NaN
Water Level (cm),0.533234,1.000000,0.938879,NaN
Discharges (m3/s),0.389201,0.938879,1.000000,NaN
dayofseason,NaN,NaN,NaN,NaN


In [ ]:
df_norm = (df - df.min()) / (df.max() - df.min())

df_max = df.groupby("seasonyear").max()
df_max_norm = (df_max - df_max.min()) / (df_max.max() - df_max.min())
df_max_date = df.groupby(df.index.year).idxmax()
df_max_date = df_max_date.apply(lambda col: col.dt.dayofyear)